
## Importing Saved weights and files

In [ ]:
!unzip files

Dataset found here: https://www.kaggle.com/anujms/car-damage-detection

## Importing libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re
import random
from tqdm import tqdm
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import Sequential, Model, load_model
!pip install flask-ngrok
import io
from base64 import encodebytes
from PIL import Image
from flask import jsonify
import json

## Importing Saved models

In [ ]:
Stage_1_model = load_model('densenet_stage1_all-0.959.hdf5')
Stage_2_model = load_model('densenet_stage2_all-0.804.hdf5')
Stage_3_model = load_model('densenet_stage3_all-0.696.hdf5')

## Importing Darknet git

In [ ]:
!/usr/local/cuda/bin/nvcc --version
# Check if GPU is selected as hardware accelerator
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if not '/device:GPU:0' in device_name:
    print('\nERROR: GPU is not selected as hardware accelerator!')
else:
    print(device_name)

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!ls
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!make
%cd /content/

## Generating report for a given Image

In [ ]:
#Creating a single function which predicts wether Car is damaged or not and localizing the damage, severity of damage for a single img
def report(img_path):
    report_pred = []
    #Creating Train and test txt files:
    test_file = open('/content/test.txt','w')
    test_file.write('/content/'+img_path+'\n')
    test_file.close()
    
    img = load_img('/content/'+img_path,target_size = (256,256))
    #Converting into array
    img_arr = img_to_array(img)
    img_arr = img_arr.reshape((1,) + img_arr.shape)
    
    #Checking if Damaged or not
    s1_pred = Stage_1_model.predict(img_arr)
    if s1_pred <=0.5:
        report_pred.append('Damaged')
    else:
        report_pred.append('Not Damaged')
        return report_pred
    #Checking for Damage Localization
    s2_pred = Stage_2_model.predict(img_arr)
    n = np.argmax(s2_pred)
    if n == 0:
        report_pred.append('Front')
    elif n ==1:
        report_pred.append('Rear')
    else:
        report_pred.append('Side')
    
    #Checking for Damage Severity
    s3_pred = Stage_3_model.predict(img_arr)
    c = np.argmax(s3_pred)
    if c == 0:
        report_pred.append('Minor')
    elif c ==1:
        report_pred.append('Moderate')
    elif c==2:
        report_pred.append('Severe')

    image_path = '/content/'+img_path
        
    #Using YOLO to detect the damage type
    %cd darknet
    temp = !./darknet detector test /content/custom.data /content/yolov3_custom.cfg /content/yolov3_custom_4000.weights -dont_show </content/test.txt> result.txt

    predicted = load_img('predictions.jpg')
    predicted.save('/content/pred.jpg')
    
    %cd /content/

    return report_pred

In [ ]:
def get_response_image(image_path):
    pil_img = Image.open(image_path, mode='r') # reads the PIL image
    byte_arr = io.BytesIO()
    pil_img.save(byte_arr, format='PNG') # convert the PIL image to byte array
    encoded_img = encodebytes(byte_arr.getvalue()).decode('ascii') # encode as base64
    return encoded_img

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template
from werkzeug.utils import secure_filename
from flask import send_file
from flask import Flask, redirect, url_for, request, render_template
app = Flask(__name__)
   #starts ngrok when the app is run

@app.route("/", methods=["GET", "POST"])
def home():
    return render_template('app.html')

@app.route('/predict', methods=['GET', 'POST'])
def upload():
	if request.method == 'POST':
		f = request.files['file']
		filepath = os.path.join('/content/',f.filename)
		f.save(filepath)
	preds = report(f.filename)
	encoded_img = get_response_image('pred.jpg')
	if preds[0] == 'Damaged':
		jsonObject = {"msg":"The vehicle is damaged on the "+preds[1]+" suffering "+preds[2]+" damages. Below is the type of damage detected.","encodedImg":'<img src="data:image/png;base64, '+encoded_img+' alt="Red dot" style="width:400px;height:400px;" />'}
		return json.dumps(jsonObject)
	else:
		jsonObject = {"msg":"Are you sure the vehicle is damaged?. Please check once."}
		return json.dumps(jsonObject)

In [ ]:
run_with_ngrok(app)
if __name__ == "__main__":
    app.run()